データとlibをダウンロード

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id = '1J8brwnHgopc8k_il49FEAB_3BdQvdCAU'  # 共有リンクで取得した id= より後の部分
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('lib.zip')

In [4]:
ls

adc.json  lib.zip  sample_data/


In [0]:
import os
# フォルダ「data」が存在しない場合は作成する
data_dir = "./lib/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [0]:
    import zipfile

    save_path = "lib.zip"
    
    # ZIPファイルを読み込み
    zip = zipfile.ZipFile(save_path)
    zip.extractall("lib")  # ZIPを解凍
    zip.close()  # ZIPファイルをクローズ

    # ZIPファイルを消去
    os.remove(save_path)

In [0]:
id = '1eDylstW-XZ7tfPmUrG__hEsWFxTWEMZE'  # 共有リンクで取得した id= より後の部分
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('stanford-dogs-dataset.zip')

In [0]:
import os
# フォルダ「data」が存在しない場合は作成する
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [0]:
    import zipfile

    save_path = "stanford-dogs-dataset.zip"
    
    # ZIPファイルを読み込み
    zip = zipfile.ZipFile(save_path)
    zip.extractall("data")  # ZIPを解凍
    zip.close()  # ZIPファイルをクローズ

    # ZIPファイルを消去
    os.remove(save_path)

In [10]:
%cd data/images/Images

/content/data/images/Images


In [0]:
os.mkdir("./train/")
os.mkdir("./val/")

In [0]:
import glob
import shutil

foruda = os.listdir(path='.')
path = os.getcwd()  

for a in foruda:
 if a.find("-") != -1:
  files = os.listdir(path + "/" + a)  
  count = len(files)  
  files = glob.glob("./"+a +"/*")
  i = 0
  for file in files:
    i = 1 + i
    if i < count*1/4:
     data_dir = "./val/" + a
     if not os.path.exists(data_dir):
      os.mkdir(data_dir)
     shutil.move(file, './val/'+ a + "/")
    else:
     data_dir = "./train/" + a
     if not os.path.exists(data_dir):
      os.mkdir(data_dir)
     shutil.move(file, './train/'+ a + "/")
  shutil.rmtree("./"+a)

In [16]:
cd ../val

/content/data/images/Images/val


In [0]:
cd ../val

*モデルの適用*

In [21]:
%%time
! npm install -g localtunnel
!python3 -m pip install visdom
!python3 -m visdom.server -port 8076 >> visdomlog.txt 2>&1 &
!lt --port 8076 >> url.txt 2>&1 &

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.0
added 35 packages from 21 contributors in 1.714s
     |████████████████████████████████| 686kB 8.6MB/s 
     |████████████████████████████████| 204kB 38.3MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp36-none-any.whl size=655252 sha256=272b9ea8b292fe0abea940acb3fac7961180fd7918b767d8a64e4b5494196b06
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5711 sha256=bdac12a7d24716f5113bb209f3dfbac19de1a74b0a8e078e6f91a47df5d7fc36
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile
CPU times: user 97.9 ms, sys: 72 ms, total: 170 ms
Wall time: 17.4 s


In [22]:
%%time
import time
time.sleep(5)
! cat url.txt

CPU times: user 11.8 ms, sys: 16.5 ms, total: 28.3 ms
Wall time: 6.26 s


In [0]:
from lib.lib import network_finetuning as network
from lib.lib import architecture
from lib.lib import dataloader as dl
from lib.lib import optimizer
from lib.lib import trainer

import torchvision

In [24]:
%%time
net = network.ResNet50(class_size=120)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [25]:
%%time
batch_size = 128
train_dataset = torchvision.datasets.ImageFolder(root='./data/images/Images/train', transform=dl.simple_transform(resize=224))
test_dataset = torchvision.datasets.ImageFolder(root='./data/images/Images/val', transform=dl.simple_transform(resize=224))
data_loader = dl.DataLoader(train_dataset, test_dataset, batch_size=batch_size)

CPU times: user 55.6 ms, sys: 14.7 ms, total: 70.3 ms
Wall time: 72.7 ms


In [26]:
%%time
model = architecture.CNN_Architecture(net)

CPU times: user 2.25 ms, sys: 34.3 ms, total: 36.5 ms
Wall time: 76.1 ms


In [27]:
%%time
model.train(data_loader, epoch_count=50, is_inception=False)

cuda


Setting up a new session...


----------
Epoch 1/50 2020-01-21 17:59:20.368412
val Loss: 22.7406 Acc: 0.0073
----------
Epoch 2/50 2020-01-21 17:59:46.776864
train Loss: 5.3462 Acc: 0.0000
val Loss: 4.7873 Acc: 0.0107
----------
Epoch 3/50 2020-01-21 18:02:47.164645


KeyboardInterrupt: ignored